Advent of Code, Day 11

In [67]:
with open('data.csv') as file:
    lines = file.readlines()
    lines = [l.strip() for l in lines]

lines

from dataclasses import dataclass, field
import re
import operator

ops = {
    '+' : operator.add,
    '*' : operator.mul
}

@dataclass
class Monkey:
    id: int = 0
    starting_items: list[int] = field(default_factory=list)
    operator: str = ''
    operation_int: int = 0
    test_divisor: int = 0
    truthy_target_monkey_id: int = 0
    falsy_target_monkey_id: int = 0
    operator_is_item: bool = False
    inspect_count: int = 0

    def inspect_items(self):
        for idx, item in enumerate(self.starting_items):
            self.inspect_count += 1
            if self.operator_is_item:
                self.starting_items[idx] = (ops[self.operator](item, item)) // 3
            else:
                self.starting_items[idx] = (ops[self.operator](item, self.operation_int)) // 3

    def distribute_items(self):
        for idx, item in enumerate(self.starting_items):
            if self.starting_items[idx] % self.test_divisor == 0:
                target_monkey = next(x for x in monkies if x.id == self.truthy_target_monkey_id)
                target_monkey.starting_items.append(item)
            else:
                target_monkey = next(x for x in monkies if x.id == self.falsy_target_monkey_id)
                target_monkey.starting_items.append(item)

        self.starting_items = []

monkies = []

for idx, line in enumerate(lines):
    if line.startswith('Monkey'):
        monkey = Monkey()
        monkey.id = re.findall(r'\d+', line)[0]
        monkey.starting_items = list(map(int, re.findall(r'\d+', lines[idx+1])))
        monkey.operator = lines[idx+2].split(' ')[-2:-1][0]
        op_int = lines[idx+2].split(' ')[-1:][0]
        if op_int.isnumeric():
            monkey.operation_int = int(op_int)
        else:
            monkey.operator_is_item = True
        monkey.test_divisor = int(lines[idx+3].split(' ')[-1:][0])
        monkey.truthy_target_monkey_id = re.findall(r'\d+', lines[idx+4])[0]
        monkey.falsy_target_monkey_id = re.findall(r'\d+', lines[idx+5])[0]

        monkies.append(monkey)
    else:
        continue

for i in range(0, 20):
    print(f'Round {i}')
    for monkey in monkies:
        monkey.inspect_items()
        monkey.distribute_items()

top_two = sorted(monkies, key=lambda x: x.inspect_count)[-2:]

top_two[0].inspect_count * top_two[1].inspect_count

Round 0
Round 1
Round 2
Round 3
Round 4
Round 5
Round 6
Round 7
Round 8
Round 9
Round 10
Round 11
Round 12
Round 13
Round 14
Round 15
Round 16
Round 17
Round 18
Round 19


78960

For each monkey, we need to resolve it's turn with the following steps:

1. Monkey inspects an item with the initial worry level.
2. Worry level is modified by operator and operation_int.
3. Monkey gets bored with item. Worry level is divided by 3.
4. Current worry level is tested with divisor.
5. Decide where to throw item.